In [176]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Literal
from pydantic import BaseModel, Field, ValidationError
from dotenv import load_dotenv
import json

In [177]:
#load environment variables from a .env file
load_dotenv()

#initiate the language model fir structured output
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [178]:
class SentimentSchema(BaseModel):
    sentiment: Literal["positive", "negative"] = Field(description="The sentiment of the review")

class DiagnosisSchema(BaseModel):
     issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review'),
     tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user'),
     urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')


In [179]:
structured_model = model.with_structured_output(SentimentSchema)
structured_diagnosis_model = model.with_structured_output(DiagnosisSchema)

In [180]:
# prompt = 'what the the sentiment of review - The software is not'
# response = structured_model.invoke(prompt)


In [181]:
class ReviewState(TypedDict):
    review: str
    sentiment: Literal["positive", "negative"]
    diagnosis: dict
    response: str

In [182]:
#sentiment analysis method
def find_sentiment(state: ReviewState):
    prompt = f'for the following review find out the sentiment - {state["review"]}'
    response = structured_model.invoke(prompt).sentiment
    return {"sentiment": response}

#conditional edge method
def check_sentiment(state: ReviewState) -> Literal["positive_response", "run_diagnosis"]:
    if state["sentiment"] == "positive":
        return "positive_response"
    else:
        return "run_diagnosis"

#positive response method
def positive_response(state: ReviewState):
    prompt = f'write a warm thank you message in response to this review: \n\n "{state["review"]} \n and also kindly ask the user to leave the feedback on our website'

    response = model.invoke(prompt).content
    return {"response": response}

#positive response method
def run_diagnosis(state: ReviewState):
    prompt = f"""Diagnose this negative review:\n\n {state['review']} \n 
    "Return issue_type, tone, and urgency."""

    response = structured_diagnosis_model.invoke(prompt)
    
    #response conversion to dict
    return {"diagnosis": response.model_dump()}

def negative_response(state: ReviewState):

    prompt = f"""You are a support assistant.",
    The user had a '{state['diagnosis']['issue_type']}' issue, sounded '{state['diagnosis']['tone']}', and marked urgency as '{state['diagnosis']['urgency']}' Write an empathetic, helpful resolution message"""
    
    response = model.invoke(prompt).content

    return {"response": response}

In [183]:
graph = StateGraph(ReviewState)
graph.add_node('find_sentiment',find_sentiment)
graph.add_node('positive_response',positive_response)
graph.add_node('run_diagnosis',run_diagnosis)
graph.add_node('negative_response',negative_response)

graph.add_edge(START,'find_sentiment')
graph.add_edge('find_sentiment','positive_response')
graph.add_edge('find_sentiment','run_diagnosis')
graph.add_edge('positive_response',END)
graph.add_edge('run_diagnosis','negative_response')
graph.add_edge('negative_response',END)

workflow = graph.compile()


In [184]:
initial_state = {
    "review": "The app crashes frequently and the customer support was unhelpful."
}

final_state = workflow.invoke(initial_state)
print(final_state)

{'review': 'The app crashes frequently and the customer support was unhelpful.', 'sentiment': 'negative', 'diagnosis': {'issue_type': 'Bug', 'tone': 'frustrated', 'urgency': 'high'}, 'response': "Subject: We're Here to Help with Your Bug Issue\n\nHi [User's Name],\n\nI truly understand how frustrating it can be to encounter a bug, especially when it disrupts your workflow. I want to assure you that we’re here to help you resolve this as quickly as possible.\n\nCould you please provide me with a few more details about the issue? Specifically, any error messages you’re seeing, the steps you took before the bug occurred, and the device or platform you’re using would be incredibly helpful. This information will allow us to diagnose the problem more effectively.\n\nThank you for your patience, and I’m committed to getting this sorted out for you as soon as possible.\n\nBest regards,\n\n[Your Name]  \n[Your Position]  \n[Your Contact Information]  "}
